In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
 
# 读入数据
path = 'ex3data1.mat'
data = sio.loadmat(path)
print(data)
print(type(data))
print(data.keys())
raw_X = data['X']
raw_Y = data['y']
# (5000, 400)
print(raw_X.shape)
# (5000, 1)
print(raw_Y.shape)


{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011', '__version__': '1.0', '__globals__': [], 'X': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'y': array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)}
<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])
(5000, 400)
(5000, 1)


In [3]:
# 损失函数，找出最小的损失函数
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
 
#交叉熵损失函数
def Cost_Function(theta, X, y, lamda):
    A = sigmoid(X @ theta)
    first = y * np.log(A)
    second = (1 - y) * np.log(1 - A)
    reg = np.sum(np.power(theta[1:], 2)) * (lamda / (2 * len(X)))
    return -np.sum(first + second) / len(X) + reg
 
 
def gradient_reg(theta, X, y, lamda):
    reg = theta[1:] * (lamda / len(X))
    reg = np.insert(reg, 0, values=0, axis=0)#插入第一行0
    first = (X.T @ (sigmoid(X @ theta) - y)) / len(X)
    return first + reg


In [4]:
X = np.insert(raw_X, 0, values=1, axis=1)#在X中插入一列1
# (5000, 401)
print(X.shape)
y = raw_Y.flatten()#对y进行降维
# (5000,)
print(y.shape)

(5000, 401)
(5000,)


In [5]:
# 利用内置函数求最优化
from scipy.optimize import minimize
 
# K为标签个数
def one_vs_all(X, y, lamda, K):
    n = X.shape[1]#X的列数401
    theta_all = np.zeros((K, n))#（10，401），sita使用0进行随机初始化，这里也可以标准正态分布进行
   #第0列到第9列分别对应类别1到10
    for i in range(1, K + 1):#遍历到k 1-k 对应1-10
        
        theta_i = np.zeros(n, )#传入minimize的必须是一维（401，）
 
        res = minimize(fun=Cost_Function,
                       x0=theta_i,
                       args=(X, y == i, lamda),
                       method='TNC',
                       jac=gradient_reg
                       )
        theta_all[i - 1, :] = res.x #将字典中x（theta）的值赋给theta
        #[i-1,:]与索引对应（0，9）
    return theta_all
 
 
lamda = 0.1#这是一个超参数，需要我们自己调试，在这个例子中lamda=0.1要比01的结果更好
K = 10
theta_final = one_vs_all(X, y, lamda, K)
print(theta_final)

[[-3.07873081e+00  0.00000000e+00  0.00000000e+00 ...  6.82393343e-03
   2.67580726e-11  0.00000000e+00]
 [-3.72948858e+00  0.00000000e+00  0.00000000e+00 ...  2.32478221e-02
  -2.55003799e-03  0.00000000e+00]
 [-5.71290652e+00  0.00000000e+00  0.00000000e+00 ... -6.29010779e-05
  -3.61572765e-07  0.00000000e+00]
 ...
 [-9.12690376e+00  0.00000000e+00  0.00000000e+00 ... -6.15426688e-04
   6.94440176e-05  0.00000000e+00]
 [-5.62470100e+00  0.00000000e+00  0.00000000e+00 ... -1.12741229e-02
   8.58707069e-04  0.00000000e+00]
 [-8.06526810e+00  0.00000000e+00  0.00000000e+00 ... -3.49598102e-05
   9.70425814e-07  0.00000000e+00]]


In [6]:
def predict(X, theta_final):
    # (5000,401) (10,401) => (5000,10)
    h = sigmoid(X @ theta_final.T)#假设函数，输出h为1的概率
    h_argmax = np.argmax(h, axis=1)#按行返回概率最大的数字索引
    return h_argmax+1 #索引+1对应数字
 
 
y_pred = predict(X, theta_final)
acc = np.mean(y_pred == y)
# 0.9446，这里是精确度已经很高了，继续运行几次可以发现精确度的变化不大，所以比较稳定。
print(acc)

0.9646
